In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('./resources/TheLastofUsPartIIReviews_csv.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80606 entries, 0 to 80605
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    80606 non-null  int64 
 1   name          80606 non-null  object
 2   date          80606 non-null  object
 3   rating        80606 non-null  object
 4   review        80606 non-null  object
 5   thumbs up     80606 non-null  object
 6   total thumbs  80606 non-null  object
dtypes: int64(1), object(6)
memory usage: 4.3+ MB


In [9]:
df.head(100)

,Unnamed: 0,name,date,rating,review,thumbs up,total thumbs
0,0,SC4RECR00W,"Jun 19, 2020",1,I can't believe how they got rid of the main c...,320,427
1,1,Bucketanarchy,"Jun 20, 2020",0,To say I didn't like it would be insulting any...,237,365
2,2,POW15,"Jun 30, 2020",2,As someone who was a big fan of the Last of Us...,210,326
3,3,tourist27,"Oct 31, 2020",2,I bet gamers in 2100 will laugh their ass off ...,201,317
4,4,lastofreviews,"Nov 22, 2020",1,"I'll be honest now, first 20 minutes (or howev...",184,291
...,...,...,...,...,...,...,...
95,95,Asmarath,"Jun 19, 2020",4,This game has so many issues.\r The graphics a...,85,143
96,96,ny13123,"Jun 19, 2020",1,The system and graphics in this game are great...,85,143
97,97,GamingChemistry,"Sep 11, 2020",0,The complexity of the characters has been dest...,85,143
98,98,Whydo,"Sep 10, 2020",3,This game is bad. Don’t trust the critics and ...,85,143


In [10]:
import nltk
nltk.download('words')

def removeNonEnglishReviews(text):
    words = set(nltk.corpus.words.words())

    return ' '.join(w for w in nltk.wordpunct_tokenize(text) \
         if w.lower() in words or not w.isalpha())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\daniszab\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

STOPWORDS = stopwords.words('english')

def remove_stopwords(text):
    '''Stopword-ök eltávolítása'''
    tokenized_text = text.split(' ')
    return ' '.join([w for w in tokenized_text if not w in STOPWORDS])

In [ ]:
useless_words = []

with open('./resources/useless_words.txt','r') as f:
    for line in f:
        useless_words.extend(line.split())

In [ ]:
del df['Unnamed: 0']
df['review'] = df['review'].replace({'#':''}, regex = True) #eltünteti a # karakteres rekordokat
df['review'] = df['review'].str.strip() #ezután stripeljük, hogy ne maradjon spaces
df = df[df['review'].map(lambda x: x.isascii())] #eltüntetjük a nem asci karakteres szavkat (jó a ciril/kanji betűkre stb)
df = df[df['review'] != ''] #elüntetjük az üres sorokat
df['review'] = df['review'].str.lower() #kisbetűssé tesszük

In [ ]:
#Külön listába kigyűjtött szavak eltávolítása --wordcloud segítségével
useless = r'\b(?:{})\b'.format('|'.join(useless_words))
df['review'] = df['review'].replace(useless, '')
df['review'] = df['review'].str.strip()

In [ ]:
#Kevés karakteres reviewok kigyűjtése -- érdekesség
df_small = df[df['review'].str.len() < 100]
df_small.to_json('./df_small.json')
df_small.info()

In [ ]:
df['review'] = df['review'].apply(removeNonEnglishReviews)
df['review'] = df['review'].apply(remove_stopwords)
df['review'] = df['review'].replace({"'":''}, regex = True)
df['review'] = df['review'].str.strip()
df = df[df['review'] != '']

In [ ]:
df.info()

In [ ]:
df.reset_index()

In [ ]:
df['thumbs down'] = df['total thumbs'].astype(int) - df['thumbs up'].astype(int)

In [ ]:
df.to_json('./resources/preprocessed.json')